## Library

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from IPython.display import IFrame
import datetime

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding = 'latin-1')

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
#Change InvoiceDate to date type
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'])

In [6]:
# Add primary key for data
data['primary_key']=np.arange(data.shape[0])
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,primary_key
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,4
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,541904
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,541905
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,541906
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,541907


In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
data.to_sql('data', con = engine)

## Add query function
def run_query(a):
    df_sql = pd.read_sql_query(a,con=engine)
    return df_sql

## Run query to discover the dataset

#### Every month's orders quantity

In [8]:
SalePerYear_Month = """
SELECT
strftime("%Y %m",InvoiceDate) AS YEAR_Month,
COUNT(distinct(InvoiceNo)) AS NumOfOrders
FROM
data
GROUP BY
1
ORDER BY
1
"""
run_query(SalePerYear_Month)

,YEAR_Month,NumOfOrders
0,2010 12,2025
1,2011 01,1476
2,2011 02,1393
3,2011 03,1983
4,2011 04,1744
5,2011 05,2162
6,2011 06,2012
7,2011 07,1927
8,2011 08,1737
9,2011 09,2327


In [9]:
## 2011 12 has much lower NumOfOrders, let's check the last day of 2011 12

Last_day_2011_12  = """
SELECT
max(InvoiceDate)
FROM
data
where strftime("%Y %m",InvoiceDate) = '2011 12'
"""
run_query(Last_day_2011_12)

,max(InvoiceDate)
0,2011-12-09 12:50:00.000000


In [10]:
# last day is 2011-12-09 not 2011-12-31, so clean up 2011 12 records
SalePerYear_Month = """
SELECT
strftime("%Y %m",InvoiceDate) AS YEAR_Month,
COUNT(distinct(InvoiceNo)) AS NumOfOrders
FROM
data
where
YEAR_Month <> '2011 12'
GROUP BY
1
ORDER BY
1
"""
run_query(SalePerYear_Month)

,YEAR_Month,NumOfOrders
0,2010 12,2025
1,2011 01,1476
2,2011 02,1393
3,2011 03,1983
4,2011 04,1744
5,2011 05,2162
6,2011 06,2012
7,2011 07,1927
8,2011 08,1737
9,2011 09,2327


#### Every month's total revenue,large to small

In [11]:
RevPerYear_Month = """
SELECT
strftime("%Y %m",InvoiceDate) AS YEAR_Month,
sum(Quantity * UnitPrice ) as Total_Revenue
FROM
data
where
YEAR_Month <> '2011 12'
GROUP BY
1
ORDER BY
2 desc
"""
run_query(RevPerYear_Month)

,YEAR_Month,Total_Revenue
0,2011 11,1461756.250
1,2011 10,1070704.670
2,2011 09,1019687.622
3,2010 12,748957.020
4,2011 05,723333.510
5,2011 06,691123.120
6,2011 03,683267.080
7,2011 08,682680.510
8,2011 07,681300.111
9,2011 01,560000.260


#### Every month's most spend customer

In [12]:
Top3_Client_month = """
With A AS(SELECT
strftime("%Y %m",InvoiceDate) AS YEAR_Month,
sum(Quantity * UnitPrice ) as Total_Revenue,
CustomerID
FROM
data
where
YEAR_Month <> '2011 12' and CustomerID is not null
GROUP BY
1,3
ORDER BY
2 desc)

select * from (select *,ROW_NUMBER() over(partition by YEAR_Month ORDER BY Total_Revenue desc) as Rank
from A) as B
where Rank <=3
"""
run_query(Top3_Client_month)

,YEAR_Month,Total_Revenue,CustomerID,Rank
0,2010 12,27834.61,18102.0,1
1,2010 12,19950.66,15061.0,2
2,2010 12,13112.52,16029.0,3
3,2011 01,26476.68,14646.0,1
4,2011 01,22998.40,15749.0,2
5,2011 01,18620.20,17450.0,3
6,2011 02,22752.46,14646.0,1
7,2011 02,14022.92,12415.0,2
8,2011 02,10535.48,18102.0,3
9,2011 03,21462.40,14646.0,1


### get every month most revenue country 

In [13]:
Top3_Country_month = """
With A AS(SELECT
strftime("%Y %m",InvoiceDate) AS YEAR_Month,
sum(Quantity * UnitPrice ) as Total_Revenue,
Country
FROM
data
where
YEAR_Month <> '2011 12' and Country is not null
GROUP BY
1,3
ORDER BY
2 desc)

select * from (select *,ROW_NUMBER() over(partition by YEAR_Month ORDER BY Total_Revenue desc) as Rank
from A) as B
where Rank <= 3
"""
run_query(Top3_Country_month)

,YEAR_Month,Total_Revenue,Country,Rank
0,2010 12,676742.620,United Kingdom,1
1,2010 12,14562.840,Germany,2
2,2010 12,9575.360,France,3
3,2011 01,434308.300,United Kingdom,1
4,2011 01,26611.160,Netherlands,2
5,2011 01,21656.520,EIRE,3
6,2011 02,408247.910,United Kingdom,1
7,2011 02,22932.110,Netherlands,2
8,2011 02,14627.470,Australia,3
9,2011 03,559707.390,United Kingdom,1


United Kingdom always has top 1 revenue, other countries revenue is much lower than United Kingdom.

In [14]:
# check the quantity of Description
Quantity_Description  = """
SELECT
count(distinct(StockCode))
FROM
data
"""
run_query(Quantity_Description)

,count(distinct(StockCode))
0,4070


### The average revelue of top 15 best selling products

In [15]:
avg_rev_top15_bestsell  = """
SELECT StockCode,
Description,
sum(Quantity) as NumOfSales,
sum(Quantity * UnitPrice ) / count(distinct(InvoiceNo)) as AvgRevenue
FROM
data
group by StockCode
order by 3 desc
limit 15
"""
run_query(avg_rev_top15_bestsell)

,StockCode,Description,NumOfSales,AvgRevenue
0,22197,SMALL POPCORN HOLDER,56450,35.358856
1,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847,25.162833
2,85099B,JUMBO BAG RED RETROSPOT,47363,43.258094
3,85123A,WHITE HANGING HEART T-LIGHT HOLDER,38830,43.586153
4,84879,ASSORTED COLOUR BIRD ORNAMENT,36221,40.163304
5,21212,PACK OF 72 RETROSPOT CAKE CASES,36039,15.786897
6,23084,RABBIT NIGHT LIGHT,30646,65.128380
7,22492,MINI PAINT SET VINTAGE,26437,43.103641
8,22616,PACK OF 12 LONDON TISSUES,26315,15.481087
9,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24753,13.930823


#### top 10 most return products

In [16]:
top10_MostReturn  = """
SELECT StockCode,
Description,
sum(Quantity) as Retrun_Quantity,
UnitPrice,
sum(Quantity*UnitPrice) as Return_value
FROM
data
where Quantity < 0 and CustomerID is not null
group by StockCode
order by 3
limit 10
"""
run_query(top10_MostReturn)

,StockCode,Description,Retrun_Quantity,UnitPrice,Return_value
0,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2.08,-168469.60
1,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74494,1.04,-77479.64
2,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9367,0.03,-298.65
3,M,Manual,-3995,0.50,-112165.39
4,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3150,2.10,-6591.42
5,20971,PINK BLUE FELT CRAFT TRINKET BOX,-2617,1.25,-2778.77
6,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-2578,2.55,-6624.30
7,21175,GIN + TONIC DIET METAL SIGN,-2030,1.85,-3775.33
8,22920,HERB MARKER BASIL,-1527,0.55,-841.05
9,22273,FELTCRAFT DOLL MOLLY,-1447,2.95,-3512.65


In [17]:
# more details on Manual
Manual_Return = """
select * 
from
data
where StockCode = 'M' and Quantity < 0 and CustomerID is not null
order by Quantity
"""
run_query(Manual_Return)
#Manual has different Unit Price

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,primary_key
0,187609,C552995,M,Manual,-1350,2011-05-12 15:19:00.000000,0.16,18133.0,United Kingdom,187609
1,392671,C570708,M,Manual,-600,2011-10-12 10:11:00.000000,0.19,12908.0,United Kingdom,392671
2,319804,C564899,M,Manual,-480,2011-08-31 11:31:00.000000,0.22,14646.0,Netherlands,319804
3,86950,C543624,M,Manual,-200,2011-02-10 15:33:00.000000,0.41,12931.0,United Kingdom,86950
4,293438,C562601,M,Manual,-144,2011-08-08 10:09:00.000000,0.11,12901.0,United Kingdom,293438
...,...,...,...,...,...,...,...,...,...,...
170,455709,C575638,M,Manual,-1,2011-11-10 13:48:00.000000,40.46,12473.0,Germany,455709
171,523375,C580510,M,Manual,-1,2011-12-04 13:48:00.000000,1.25,14456.0,United Kingdom,523375
172,533082,C581009,M,Manual,-1,2011-12-07 09:15:00.000000,125.00,16971.0,United Kingdom,533082
173,534308,C581145,M,Manual,-1,2011-12-07 13:48:00.000000,9.95,17490.0,United Kingdom,534308


In [18]:
Order_Rev = """
with a as(
SELECT
InvoiceNo,
sum(Quantity * UnitPrice ) as Total_Revenue
FROM
data
GROUP BY
1
ORDER BY
2 desc
),
b as(
select case
    when Total_Revenue >= 1000 then 'Large Order'
    when Total_Revenue >= 100 and Total_Revenue < 1000 then 'Normal Order'
    when Total_Revenue > 0 and Total_Revenue < 100 then 'Small Order'
    when Total_Revenue = 0 then 'Free Order'
    else 'Return Order'
end as OrderGroup,
InvoiceNo,
Total_Revenue
from a
),
c as (
select OrderGroup,
    count(InvoiceNo) as Num_Order,
    (100*count(InvoiceNo))/(select count(distinct(InvoiceNo)) from data) as Perc_Orders,
    sum(Total_Revenue) as Total_Revenue
from b
group by 1)

select * from c order by Num_Order desc
"""

run_query(Order_Rev)

,OrderGroup,Num_Order,Perc_Orders,Total_Revenue
0,Normal Order,15397,59,5426436.692
1,Return Order,3838,14,-918936.610
2,Small Order,2721,10,116642.781
3,Free Order,2102,8,0.000
4,Large Order,1842,7,5123605.071


59% of orders are in range 100 to 1000.

Return orders have 14% , should take more strategies to decrease the percentile of Return orders.

Large Order has great developing potntial that 7% orders but has similiar Total_Revenue with Normal order. 

## Data Visualizing with Tableau

In [19]:
%%html
<div class='tableauPlaceholder' id='viz1658420527894' style='position: relative'><noscript><a href='#'><img alt='UK retailer E-Commerce Sales Performance ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;9P&#47;9PHQ48HF8&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;9PHQ48HF8' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;9P&#47;9PHQ48HF8&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1658420527894');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1300px';vizElement.style.height='1227px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1300px';vizElement.style.height='1227px';} else { vizElement.style.width='100%';vizElement.style.height='1427px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>